**User input categories**
* Product
* Return reason
* Brand
* Timeline
* Claimed item condition

In [1]:
# For this demonstration, I will go ahead and make up answers for each of the user input categories to show proof-of-concept

product  = "topwear"
return_reason = "defective"
brand = "uniqlo"
timeline = "within 30 days"
item_condition = "gently used"

Allow the user to upload multiple images. 

Prompt engineer Gemini to return 1 of 3 final categories: **Return to store, donate, recycle**

In [2]:
from IPython.display import display
from ipywidgets import FileUpload
from PIL import Image
import io

upload_widget = FileUpload(accept='image/*', multiple=True)
display(upload_widget)

uploaded_images = {}

def handle_upload(change):
    global uploaded_images
    uploaded_images.clear()  # Reset previous uploads
    for fileinfo in upload_widget.value:
        filename = fileinfo["name"]
        image_data = fileinfo["content"]
        image = Image.open(io.BytesIO(image_data))
        uploaded_images[filename] = image
        display(image)

upload_widget.observe(handle_upload, names='value')


FileUpload(value=(), accept='image/*', description='Upload', multiple=True)

In [3]:
from google import genai
from google.genai import types
import io
import os
from dotenv import load_dotenv

load_dotenv()
gemini_key = os.getenv("GEMINI_API_KEY")

if not uploaded_images:
    raise ValueError("No images uploaded yet. Please upload images first.")

# Convert the images into the correct format (types.Part)
gemini_images = []

for filename, pil_image in uploaded_images.items():
    if pil_image.mode == "RGBA":
        pil_image = pil_image.convert("RGB")
    
    img_bytes_io = io.BytesIO()
    
    # Convert PIL Image to a byte stream
    pil_image.save(img_bytes_io, format="JPEG")  
    img_bytes_io.seek(0)  
    
    # Convert to types.Part for Gemini
    gemini_images.append(
        types.Part.from_bytes(
            data=img_bytes_io.getvalue(),
            mime_type="image/jpeg"
        )
    )

client = genai.Client(api_key=gemini_key)
response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=[f"""
    Your job is to assess the product condition based on the images provided into 3 categories: 1. Return to store; 2: Recycle; 3: Donate. The customer wants to return a {product} from {brand} because it is {return_reason}. 
    The item is {item_condition} and they purchased it {timeline}. Categorize them and if the images are ambiguous, say so.
    """] + gemini_images
)


print(response.text)

response_numeric = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=[f"""
    Your job is to assess the product condition based on the images provided into 3 categories: 1. Return to store; 2: Recycle; 3: Donate. The customer wants to return a {product} from {brand} because it is {return_reason}. 
    The item is {item_condition} and they purchased it {timeline}. ONLY output the number for the category. If it is ambiguous, output 4.
    """] + gemini_images
)

print(response_numeric.text)

Okay, I have reviewed the image and the crops provided. Here's my assessment based on the information given:

*   **Return to Store:** Based on the image, the shirt appears wrinkled. It is hard to ascertain the damage on the shirt and needs closer examination to confirm if the shirt is defective.

Since the customer is claiming the item is defective and the return is within the 30-day window, a return to the store would be the best option. The store can then make a final determination about the defect.


1


**OpenRouteService API connection**

* If categorization is between 1-3, then establish openrouteservice API connection to allow users to select a pick-up time and driver.
* If categorization is 4, then print error message saying that the item is ineligible for return.

In [20]:
import requests
import json
from pprint import pprint
from geopy.geocoders import Nominatim
import folium
import openrouteservice

ors_key = os.getenv("OPENROUTESERVICE_API_KEY")

if response_numeric.text.strip() in {"1", "2", "3"}:

    geolocator = Nominatim(user_agent="geo_finder")
    user_location = input("Enter a location: ")
    location = geolocator.geocode(user_location)

    if location:
        longi = location.longitude
        lati = location.latitude
        print(f"Address: {location.address}")
        print(f"Coordinates: ({location.latitude}, {location.longitude})")
    else:
        print("Location not found. Please enter a real location.")

    # with the start and end location(s), we are now ready to connect to OpenRouteService

    your_location = [longi, lati]   
    simulated_location1 = [-79.3906, 43.6582]  # example available driver location #1
    simulated_location2 = [-79.3807, 43.6544]  # example available driver location #2
    simulated_location3 = [-79.3948, 43.6677]  # example available driver location #3

    url = "https://api.openrouteservice.org/v2/directions/driving-car"

    body = {
    "coordinates": [
        your_location,   
        simulated_location1,
        simulated_location2,  
        simulated_location3 
    ],
    "units": "m"
    }
    
    headers = {
        'Accept': 'application/json, application/geo+json',
        'Authorization': ors_key,
        'Content-Type': 'application/json'
    }
    
    # API request
    call = requests.post(url, json=body, headers=headers)
    
    # Print response
    if call.status_code == 200:
        response_data = call.json()
    
        # Extract and print total distance & duration
        total_distance = response_data["routes"][0]["summary"]["distance"]  # Meters
        total_duration = response_data["routes"][0]["summary"]["duration"]  # Seconds
    
        print(f"Total Distance: {total_distance/1000:.2f} km")  # Convert meters to km
        print(f"Total Duration: {total_duration/60:.2f} minutes")  # Convert seconds to minutes

        segments = response_data["routes"][0]["segments"]

        for i, segment in enumerate(segments):
            segment_distance = segment["distance"] / 1000  # Convert meters to km
            segment_duration = segment["duration"] / 60  # Convert seconds to minutes
            print(f"📍 Segment {i + 1}: {segment_distance:.2f} km, ⏳ {segment_duration:.2f} minutes")

        # Decode polyline geometry to get coordinates
        route_geometry = response_data["routes"][0]["geometry"]
        route_coordinates = openrouteservice.convert.decode_polyline(route_geometry)["coordinates"]
        
        # Define all four locations (start and intermediate points)
        locations = [
            your_location,  # Start point
            simulated_location1,  # First intermediate point
            simulated_location2,  # Second intermediate point
            simulated_location3   # End point
        ]
        
        # Create a folium map centered around the starting location
        map_center = [your_location[1], your_location[0]]  # lat, lon 
        route_map = folium.Map(location=map_center, zoom_start=12)
        
        # Add the main route polyline to the map
        folium.PolyLine(route_coordinates, color="blue", weight=15, opacity=1.0).add_to(route_map)
        
        colors = ["green", "orange", "red", "blue"]
        labels = ["Your Location", "Driver 1", "Driver 2", "Driver 3"]
        
        for i, location in enumerate(locations):
            folium.Marker(
                location=[location[1], location[0]],
                popup=labels[i],
                icon=folium.Icon(color=colors[i])
            ).add_to(route_map)
        
        # Draw paths from each intermediate point to the starting point
        for loc in [simulated_location1, simulated_location2, simulated_location3]:
            body = {
                "coordinates": [
                    loc,   
                    your_location  
                ],
                "units": "m"
            }

            # Make the API request for the specific path
            call = requests.post(url, json=body, headers=headers)

            # Process the response for each route
            if call.status_code == 200:
                response_data = call.json()
                route_geometry = response_data["routes"][0]["geometry"]
                route_coordinates = openrouteservice.convert.decode_polyline(route_geometry)["coordinates"]
                
                folium.PolyLine(route_coordinates, color="purple", weight=15, opacity=1.0).add_to(route_map)
                
                # Extract and display the distance and duration for the path
                distance = response_data["routes"][0]["summary"]["distance"] / 1000
                duration = response_data["routes"][0]["summary"]["duration"] / 60
                print(f"Distance from {loc} to Start: {distance:.2f} km")
                print(f"Duration from {loc} to Start: {duration:.2f} minutes")

                folium.Marker(
                    location=[loc[1], loc[0]],  # lat, lon
                    popup=f"Distance: {distance:.2f} km, Time: {duration:.2f} mins",
                    icon=folium.Icon(color="purple")
                ).add_to(route_map)

        # Save the map to an HTML file
        route_map.save("route_map_with_paths.html")
        
    else:
        print("Error:", call.text)
    
else:
    print("Based on the uploaded images, your items are ineligible for pick-up.")

Enter a location:  uoft


Address: Second Cup UofT, 100, St George Street, University, University—Rosedale, Toronto, Golden Horseshoe, Ontario, M5S 2E5, Canada
Coordinates: (43.6625562, -79.3988018)
Total Distance: 4.87 km
Total Duration: 10.74 minutes
📍 Segment 1: 1.17 km, ⏳ 3.21 minutes
📍 Segment 2: 1.35 km, ⏳ 2.69 minutes
📍 Segment 3: 2.36 km, ⏳ 4.84 minutes
Distance from [-79.3906, 43.6582] to Start: 1.41 km
Duration from [-79.3906, 43.6582] to Start: 3.67 minutes
Distance from [-79.3807, 43.6544] to Start: 2.50 km
Duration from [-79.3807, 43.6544] to Start: 5.64 minutes
Distance from [-79.3948, 43.6677] to Start: 1.04 km
Duration from [-79.3948, 43.6677] to Start: 2.77 minutes


In [ ]:
# geolocator = Nominatim(user_agent="geo_finder")
# user_location = input("Enter a location: ")
# location = geolocator.geocode(user_location)

# if location:
#     longi = location.longitude
#     lati = location.latitude
#     print(f"Address: {location.address}")
#     print(f"Coordinates: ({location.latitude}, {location.longitude})")
# else:
#     print("Location not found. Please enter a real location.")

# # Coordinates: [longitude, latitude] format
# start_location = [-79.6425, 43.5930] 
# end_location = [longi, lati]  

Address: Second Cup UofT, 100, St George Street, University, University—Rosedale, Toronto, Golden Horseshoe, Ontario, M5S 2E5, Canada
Coordinates: (43.6625562, -79.3988018)
